In [ ]:
import wave
import numpy as np
import pylab as plt
from scipy.signal import firwin
%matplotlib inline
np.set_printoptions(precision=4)

In [ ]:
cMatrix = [
-8.0637536579e-19, -1.3147051387e-03,  7.6108568498e-03, -2.5651021426e-02,  6.8424554212e-02, -1.7058731625e-01,  6.2151763175e-01,  6.2151763175e-01, -1.7058731625e-01,  6.8424554212e-02, -2.5651021426e-02,  7.6108568498e-03, -1.3147051387e-03, -6.7593229191e-19, 
 4.2231062201e-04, -7.3849128016e-03,  2.5640816965e-02, -6.6353440284e-02,  1.5092479216e-01, -3.2077802693e-01,  1.4888931696e+00, -1.4427957650e+00,  2.3570009308e-01, -9.4794181655e-02,  4.2345341839e-02, -1.6409871256e-02,  4.5783425462e-03,  1.1331118064e-05, 
-2.1039703946e-04,  2.5136661122e-02, -1.0872200064e-01,  3.2013962622e-01, -7.8318559530e-01,  1.5750766194e+00, -1.4888752992e+00,  1.8849828001e-01,  3.9656040521e-01, -1.5899187363e-01,  4.0735786821e-02, -1.3418581423e-03, -4.7902599462e-03, -3.0094962531e-05, 
-1.5266187212e-03, -8.8261863321e-03,  4.9819305402e-02, -1.5971061030e-01,  3.9324893267e-01, -4.6219364451e-01, -1.7870407362e-05,  4.6219253697e-01, -3.9324862783e-01,  1.5971047964e-01, -4.9819250384e-02,  8.8261674102e-03,  1.5266225387e-03,  1.8763844467e-05,     
]
NTAPS = 14
ORDER = 3

In [ ]:
def sample_to_int(data):
        d = int.from_bytes(data, byteorder='little', signed=True)
        return d

def int_to_sample(data):
        d = int(data)
        return d.to_bytes(3, byteorder='little', signed=True)

In [ ]:
out_sample = 0
sample_inc = 1.001

In [ ]:
fname = "sine.wav"
f = wave.open(fname, 'rb')

f_out = wave.open("wave_out.wav", "wb")
f_out.setframerate(48000)
f_out.setsampwidth(3)
f_out.setnchannels(1)

In [ ]:
audio_delay_line = np.zeros(NTAPS)
delay_line_sample = -NTAPS

In [ ]:
def c(ixTap, ixPower):
    return cMatrix[ixPower * (NTAPS) + ixTap]*0.99

In [ ]:
def get_imp(tap, frac):
    horner_sum = c(tap, ORDER)
    for i in range(ORDER-1, -1, -1,):
        horner_sum *= frac
        horner_sum += c(tap, i)
    return horner_sum

In [ ]:
def get_c(frac):
    return [get_imp(i, frac) for i in range(NTAPS)]

In [ ]:
def process_sample(frac):
    fir_c = get_c(frac)
    v = np.sum(audio_delay_line*fir_c)
    return v

In [ ]:
while(True):
    data = f.readframes(1)
    audio_delay_line = np.roll(audio_delay_line, 1)
    audio_delay_line[0] = sample_to_int(data)
    delay_line_sample += 1
    if(len(data)==0):
        break
    while(out_sample < delay_line_sample):
        frac = out_sample % 1
        v = process_sample(frac)
        f_out.writeframes(int_to_sample(v))
        out_sample += sample_inc

In [ ]:
f.close()
f_out.close()